In [ ]:
# 1. Erstelle die Ordnerstruktur
import os
os.makedirs("sql_project/scripts", exist_ok=True)
os.makedirs("sql_project/configs", exist_ok=True)
os.makedirs("sql_project/outputs", exist_ok=True)

# 2. Wechsel in das Projektverzeichnis
%cd sql_project

In [ ]:
%%writefile requirements.txt
torch
transformers
peft
bitsandbytes
trl
accelerate
datasets
huggingface_hub

In [ ]:
%pip install -q -r requirements.txt

In [ ]:
%%writefile scripts/train.py
import os
import torch
import argparse
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from huggingface_hub import login, HfApi

def main(args):
    # Hugging Face Hub Login
    if args.push_to_hub:
        print("Initialisiere Hugging Face Hub...")
        try:
            # Try to login - if token is provided via environment or file, it will use that
            # Otherwise, it will prompt for login
            hf_token = os.environ.get("HF_TOKEN") or args.hf_token
            if hf_token:
                login(token=hf_token)
                print("✓ Erfolgreich bei Hugging Face angemeldet!")
            else:
                print("Bitte melden Sie sich bei Hugging Face an...")
                login()  # This will prompt for token or use existing credentials
                print("✓ Erfolgreich bei Hugging Face angemeldet!")
        except Exception as e:
            print(f"⚠️ Hugging Face Login Fehler: {e}")
            print("⚠️ Adapter wird nur lokal gespeichert.")
            args.push_to_hub = False
    
    # Wandb Setup - Automatically use existing account (option 2)
    # Set environment to use existing credentials
    os.environ["WANDB_MODE"] = "online"
    # If wandb is not already logged in, this will use existing credentials from ~/.netrc or environment
    try:
        import wandb
        # Check if wandb is already initialized - use a safer check
        try:
            current_run = wandb.run
        except:
            current_run = None
        
        if current_run is None:
            wandb.init(project="sql-assistant", mode="online")
    except Exception as e:
        print(f"Wandb initialization note: {e}")
        # Continue without wandb if there's an issue
    # 1. Datensatz laden
    print(f"Lade Datensatz: {args.dataset_name}")
    dataset = load_dataset(args.dataset_name, split="train")
    # Nur für Demo-Zwecke verkürzen, falls gewünscht
    if args.max_samples:
        dataset = dataset.select(range(args.max_samples))
    
    # 2. Modell & Tokenizer laden (4-bit QLoRA)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    
    print(f"Lade Modell: {args.model_name}")
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name, quantization_config=bnb_config, device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    tokenizer.pad_token = tokenizer.eos_token

    # 3. LoRA Config - für Qwen2.5 Modelle
    peft_config = LoraConfig(
        r=16, 
        lora_alpha=16, 
        lora_dropout=0.05, 
        bias="none", 
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    )
    
    # 4. Model für k-bit training vorbereiten und PEFT anwenden
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)

    # 5. Training Arguments - Kombiniere TrainingArguments mit SFTConfig
    training_args = SFTConfig(
        output_dir=args.output_dir,
        per_device_train_batch_size=args.batch_size,
        gradient_accumulation_steps=2,
        learning_rate=args.lr,
        logging_steps=10,
        num_train_epochs=args.epochs,
        fp16=False,  # Disable fp16 to avoid BFloat16 gradient scaler issue
        optim="paged_adamw_32bit",
        save_strategy="epoch",
        save_total_limit=1,
        remove_unused_columns=False,
        dataset_text_field="text",
        max_length=512,
        packing=False,
        max_grad_norm=1.0  # Gradient clipping
    )

    # 6. Daten formatieren (Qwen Template)
    def format_prompt(sample):
        prompt = f"<|im_start|>system\nYou are a SQL expert.<|im_end|>\n<|im_start|>user\n{sample['context']}\nQuestion: {sample['question']}<|im_end|>\n<|im_start|>assistant\n{sample['answer']}<|im_end|>"
        return {"text": prompt}

    train_dataset = dataset.map(format_prompt, remove_columns=dataset.column_names)

    # 7. Trainer Starten (ohne peft_config, da Modell bereits PEFT-wrapped ist)
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        processing_class=tokenizer,
        args=training_args
    )

    print("Starte Training...")
    trainer.train()
    
    print(f"Speichere Adapter nach {args.output_dir}...")
    trainer.model.save_pretrained(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)
    
    # Upload to Hugging Face Hub
    if args.push_to_hub:
        print(f"\n📤 Lade Adapter auf Hugging Face hoch: {args.hf_repo_id}")
        try:
            # Push adapter to Hub
            trainer.model.push_to_hub(
                args.hf_repo_id,
                private=args.private_repo,
                token=os.environ.get("HF_TOKEN") or args.hf_token
            )
            print(f"✓ Adapter erfolgreich hochgeladen: https://huggingface.co/{args.hf_repo_id}")
            
            # Also push tokenizer
            tokenizer.push_to_hub(
                args.hf_repo_id,
                private=args.private_repo,
                token=os.environ.get("HF_TOKEN") or args.hf_token
            )
            print("✓ Tokenizer erfolgreich hochgeladen!")
        except Exception as e:
            print(f"⚠️ Fehler beim Hochladen auf Hugging Face: {e}")
            print("⚠️ Adapter wurde lokal gespeichert.")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    parser.add_argument("--dataset_name", type=str, default="b-mc2/sql-create-context")
    parser.add_argument("--output_dir", type=str, default="./outputs/final_model")
    parser.add_argument("--batch_size", type=int, default=4)
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--lr", type=float, default=2e-4)
    parser.add_argument("--max_samples", type=int, default=500) # Klein halten für Test
    parser.add_argument("--push_to_hub", action="store_true", help="Upload adapter to Hugging Face Hub")
    parser.add_argument("--hf_repo_id", type=str, default=None, help="Hugging Face repo ID (e.g., 'username/model-name')")
    parser.add_argument("--hf_token", type=str, default=None, help="Hugging Face token (or set HF_TOKEN env var)")
    parser.add_argument("--private_repo", action="store_true", help="Make the Hugging Face repo private")
    
    args = parser.parse_args()
    
    # Validate Hugging Face arguments
    if args.push_to_hub and not args.hf_repo_id:
        raise ValueError("--hf_repo_id is required when --push_to_hub is set")
    
    main(args)

In [ ]:
%%writefile scripts/evaluate.py
import torch
import argparse
from tqdm import tqdm
from datasets import load_dataset
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

def normalize_sql(query):
    """Bereinigt SQL von Leerzeichen und Groß/Kleinschreibung für fairen Vergleich"""
    if not query: return ""
    query = query.lower().replace(";", "").replace("\n", " ")
    return " ".join(query.split())

def main(args):
    # 1. Basis-Modell & Adapter laden
    print(f"Lade Basis-Modell: {args.base_model_name}")
    base_model = AutoModelForCausalLM.from_pretrained(
        args.base_model_name, device_map="auto", torch_dtype=torch.float16
    )
    
    print(f"Lade Adapter: {args.adapter_path}")
    model = PeftModel.from_pretrained(base_model, args.adapter_path)
    tokenizer = AutoTokenizer.from_pretrained(args.base_model_name)

    # 2. Test-Daten laden (Die letzten 100 Zeilen des Datasets als Testset nehmen)
    dataset = load_dataset("b-mc2/sql-create-context", split="train")
    test_dataset = dataset.select(range(len(dataset)-args.num_samples, len(dataset)))

    correct_count = 0
    total_count = 0

    print(f"Starte Evaluation auf {args.num_samples} Beispielen...")

    for sample in tqdm(test_dataset):
        # Prompt bauen
        messages = [
            {"role": "system", "content": "You are a SQL expert."},
            {"role": "user", "content": f"{sample['context']}\nQuestion: {sample['question']}"}
        ]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generieren
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=100)
        
        # Antwort extrahieren
        generated_full = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_sql = generated_full.split("assistant")[-1].strip()
        
        # Vergleichen (Normalized Exact Match)
        truth_norm = normalize_sql(sample["answer"])
        pred_norm = normalize_sql(generated_sql)

        if truth_norm == pred_norm:
            correct_count += 1
        total_count += 1

    accuracy = (correct_count / total_count) * 100
    print(f"\n==========================================")
    print(f"RESULTAT: Exact Match Accuracy: {accuracy:.2f}%")
    print(f"==========================================")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_model_name", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    parser.add_argument("--adapter_path", type=str, required=True)
    parser.add_argument("--num_samples", type=int, default=50)
    args = parser.parse_args()
    main(args)

In [ ]:
# Wandb Setup - Use existing account (option 2) non-interactively
import os
# Set wandb to use existing account without prompt
os.environ["WANDB_MODE"] = "online"
# If you have a wandb API key, you can set it here to avoid prompts:
os.environ["WANDB_API_KEY"] = ""


In [ ]:
# Wir trainieren das Modell und speichern es im Ordner 'outputs/v1'
# Automatisch Option 2 (Use existing W&B account) auswählen
import subprocess
import sys
import time

def run_with_auto_input(push_to_hub=False, hf_repo_id=None, hf_token=None, private_repo=False):
    """Run training script and automatically send '2' when wandb prompts"""
    cmd = [sys.executable, "scripts/train.py",
         "--model_name", "Qwen/Qwen2.5-1.5B-Instruct",
         "--output_dir", "./outputs/v1",
         "--epochs", "1",
         "--max_samples", "500"]
    
    # Add Hugging Face upload arguments if specified
    if push_to_hub:
        cmd.extend(["--push_to_hub"])
        if hf_repo_id:
            cmd.extend(["--hf_repo_id", hf_repo_id])
        if hf_token:
            cmd.extend(["--hf_token", hf_token])
        if private_repo:
            cmd.extend(["--private_repo"])
    
    process = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    buffer = ""
    sent_input = False
    
    while True:
        # Read available output
        output = process.stdout.read(1)
        if not output:
            if process.poll() is not None:
                break
            time.sleep(0.1)
            continue
            
        buffer += output
        print(output, end='', flush=True)
        
        # Check for wandb prompt and send "2" once
        if "Enter your choice:" in buffer and not sent_input:
            time.sleep(0.2)
            process.stdin.write("2\n")
            process.stdin.flush()
            sent_input = True
            buffer = ""
        
        # Keep buffer size manageable
        if len(buffer) > 500:
            buffer = buffer[-200:]
    
    return process.wait()

# Training OHNE Hugging Face Upload (Standard)
run_with_auto_input()

# Training MIT Hugging Face Upload (auskommentiert - bitte aktivieren und anpassen)
# run_with_auto_input(
#     push_to_hub=True,
#     hf_repo_id="your-username/your-model-name",  # Z.B. "manuelaschrittwieser99/qwen2.5-1.5b-sql-adapter"
#     hf_token=None,  # Optional: Token direkt übergeben, oder setzen Sie HF_TOKEN als Umgebungsvariable
#     private_repo=False  # True = privates Repo, False = öffentliches Repo
# )
    
    
    

In [ ]:
# Upload eines bereits trainierten Adapters zu Hugging Face
import torch
from huggingface_hub import login, HfApi
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

def upload_adapter_to_hub(
    adapter_path="./outputs/v1",
    hf_repo_id=None,
    base_model_name="Qwen/Qwen2.5-1.5B-Instruct",
    hf_token=None,
    private_repo=False
):
    """
    Lädt einen bereits trainierten Adapter und lädt ihn auf Hugging Face hoch.
    
    Args:
        adapter_path: Pfad zum lokal gespeicherten Adapter
        hf_repo_id: Hugging Face Repo ID (z.B. "username/model-name")
        base_model_name: Name des Basis-Modells
        hf_token: Hugging Face Token (optional, kann auch als HF_TOKEN env var gesetzt werden)
        private_repo: Ob das Repo privat sein soll
    """
    
    if not hf_repo_id:
        raise ValueError("hf_repo_id ist erforderlich (z.B. 'username/model-name')")
    
    # Login zu Hugging Face
    print("Initialisiere Hugging Face Hub...")
    token = hf_token or os.environ.get("HF_TOKEN")
    if token:
        login(token=token)
        print("✓ Erfolgreich bei Hugging Face angemeldet!")
    else:
        print("Bitte melden Sie sich bei Hugging Face an...")
        login()
        print("✓ Erfolgreich bei Hugging Face angemeldet!")
    
    # Lade Basis-Modell (nur zum Push benötigt, wird nicht geladen)
    print(f"\n📤 Lade Adapter von: {adapter_path}")
    
    # Für PEFT-Modelle müssen wir das Basis-Modell laden, um den Adapter zu pushen
    # Aber wir können es im 4-bit Modus laden, um Speicher zu sparen
    try:
        from transformers import BitsAndBytesConfig
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
        print(f"Lade Basis-Modell: {base_model_name}")
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name, 
            quantization_config=bnb_config, 
            device_map="auto"
        )
    except Exception as e:
        print(f"⚠️ Konnte Modell nicht im 4-bit Modus laden: {e}")
        print("Versuche ohne Quantisierung...")
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            device_map="auto",
            torch_dtype=torch.float16
        )
    
    # Lade Adapter
    print(f"Lade Adapter von: {adapter_path}")
    model = PeftModel.from_pretrained(base_model, adapter_path)
    
    # Lade Tokenizer
    print("Lade Tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    
    # Upload Adapter
    print(f"\n📤 Lade Adapter auf Hugging Face hoch: {hf_repo_id}")
    try:
        model.push_to_hub(
            hf_repo_id,
            private=private_repo,
            token=token
        )
        print(f"✓ Adapter erfolgreich hochgeladen: https://huggingface.co/{hf_repo_id}")
        
        # Upload Tokenizer
        tokenizer.push_to_hub(
            hf_repo_id,
            private=private_repo,
            token=token
        )
        print("✓ Tokenizer erfolgreich hochgeladen!")
        
        print(f"\n✅ Fertig! Adapter ist verfügbar unter: https://huggingface.co/{hf_repo_id}")
        
    except Exception as e:
        print(f"⚠️ Fehler beim Hochladen: {e}")
        raise

# BEISPIEL-VERWENDUNG:
# Bitte passen Sie die Parameter an:

# upload_adapter_to_hub(
#     adapter_path="./outputs/v1",  # Pfad zu Ihrem trainierten Adapter
#     hf_repo_id="your-username/your-model-name",  # Ihr Hugging Face Repo
#     base_model_name="Qwen/Qwen2.5-1.5B-Instruct",  # Basis-Modell
#     hf_token=None,  # Optional: Token hier angeben, oder HF_TOKEN env var setzen
#     private_repo=False  # True für privates Repo
# )

print("✓ Upload-Funktion bereit!")
print("\nZum Hochladen, führen Sie folgenden Code aus:")
print("upload_adapter_to_hub(")
print('    adapter_path="./outputs/v1",')
print('    hf_repo_id="your-username/your-model-name",')
print('    private_repo=False')
print(")")


In [ ]:
%%writefile scripts/deploy.py
import argparse
from huggingface_hub import HfApi, create_repo

def main(args):
    api = HfApi()
    
    # 1. Repo-Namen bauen
    full_repo_id = f"{args.username}/{args.repo_name}"
    print(f"Ziel-Repository: {full_repo_id}")

    # 2. Repository erstellen (falls es noch nicht existiert)
    try:
        create_repo(full_repo_id, repo_type="model", exist_ok=True)
        print("Repository gefunden oder erstellt.")
    except Exception as e:
        print(f"Hinweis beim Repo-Erstellen: {e}")

    # 3. Dateien hochladen
    print(f"Lade Ordner '{args.model_dir}' hoch... Bitte warten.")
    
    api.upload_folder(
        folder_path=args.model_dir,
        repo_id=full_repo_id,
        repo_type="model",
        commit_message=f"Upload model from production script: {args.repo_name}"
    )
    
    print("\n✅ Upload erfolgreich!")
    print(f"Dein Modell ist hier: https://huggingface.co/{full_repo_id}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--username", type=str, required=True, help="Hugging Face Nutzername")
    parser.add_argument("--repo_name", type=str, required=True, help="Name für das neue Modell auf HF")
    parser.add_argument("--model_dir", type=str, default="./outputs/final_model", help="Lokaler Pfad zum Modell")
    
    args = parser.parse_args()
    main(args)

In [ ]:
!python scripts/run_agent.py --adapter "manuelaschrittwieser/Qwen2.5-SQL-Assistant-Prod"

In [ ]:
!python scripts/deploy.py \
    --username "manuelaschrittwieser" \
    --repo_name "Qwen2.5-SQL-Assistant-Prod" \
    --model_dir "./outputs/v1/"

In [ ]:
%pip install -q torch transformers peft accelerate

In [ ]:
%%writefile scripts/setup_db.py
import sqlite3
import os

def create_dummy_db(db_path="data/dummy_database.db"):
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Tabelle erstellen
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        department TEXT,
        salary INTEGER,
        hire_date DATE
    )
    """)

    # Daten einfügen
    employees = [
        (1, 'Alice Smith', 'Sales', 55000, '2021-01-15'),
        (2, 'Bob Jones', 'Engineering', 85000, '2020-03-10'),
        (3, 'Charlie Brown', 'Sales', 48000, '2022-06-23'),
        (4, 'Diana Prince', 'Engineering', 92000, '2019-11-05'),
        (5, 'Evan Wright', 'HR', 45000, '2021-09-30')
    ]
    
    cursor.executemany('INSERT OR IGNORE INTO employees VALUES (?,?,?,?,?)', employees)
    conn.commit()
    conn.close()
    print(f"✅ Datenbank erstellt: {db_path}")

if __name__ == "__main__":
    create_dummy_db()

In [1]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

# ==========================================
# KONFIGURATION
# ==========================================
# 1. Hugging Face Token ein (mit "Write"-Rechten!):
HF_TOKEN = "YOUR_HF_TOKEN_HERE" 

# Namen der Modelle
BASE_MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
ADAPTER_ID = "manuelaschrittwieser/Qwen2.5-SQL-Assistant-Prod"
NEW_FULL_MODEL_ID = "manuelaschrittwieser/Qwen2.5-SQL-Assistant-Full"

# ==========================================
# AUTHENTIFIZIERUNG
# ==========================================
print(f"Logge ein mit Token...")
login(token=HF_TOKEN)

# ==========================================
# 1. LADEN (Base + Adapter)
# ==========================================
print("Lade Basis-Modell (dies kann kurz dauern)...")
# Hinweis: Auf CPU ist float32 sicherer als float16, um Abstürze zu vermeiden.
# Falls RAM-Probleme, wechsele zurück auf torch.float16
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="cpu",
    torch_dtype=torch.float16, 
    token=HF_TOKEN
)

print("Lade Adapter...")
model = PeftModel.from_pretrained(
    base_model, 
    ADAPTER_ID, 
    token=HF_TOKEN
)

# ==========================================
# 2. MERGEN
# ==========================================
print("Starte Merge-Vorgang...")
merged_model = model.merge_and_unload()

# ==========================================
# 3. SPEICHERN & UPLOAD
# ==========================================
print("Lade Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, token=HF_TOKEN)

print(f"Lade Full-Model hoch zu: {NEW_FULL_MODEL_ID} ...")

# Upload des Modells
merged_model.push_to_hub(NEW_FULL_MODEL_ID, token=HF_TOKEN)

# Upload des Tokenizers
tokenizer.push_to_hub(NEW_FULL_MODEL_ID, token=HF_TOKEN)

print("✅ Erfolgreich hochgeladen!")

Logge ein mit Token...
Lade Basis-Modell (dies kann kurz dauern)...


`torch_dtype` is deprecated! Use `dtype` instead!


Lade Adapter...
Starte Merge-Vorgang...
Lade Tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Lade Full-Model hoch zu: manuelaschrittwieser/Qwen2.5-SQL-Assistant-Full ...


...

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpik19yg4d/tokenizer.json:  73%|#######2  | 8.30MB / 11.4MB            

✅ Erfolgreich hochgeladen!


In [ ]:
%%writefile scripts/run_agent.py
import sqlite3
import torch
import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

class SQLAgent:
    def __init__(self, base_model_id, adapter_id, db_path):
        self.db_path = db_path
        print("🤖 Lade das Gehirn des Agenten...")
        
        # Modell laden
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_id)
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_id, 
            device_map="auto", 
            torch_dtype=torch.float16
        )
        self.model = PeftModel.from_pretrained(base_model, adapter_id)
        
    def generate_sql(self, question, schema_context):
        messages = [
            {"role": "system", "content": "You are a SQL expert."},
            {"role": "user", "content": f"{schema_context}\nQuestion: {question}"}
        ]
        prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_new_tokens=100)
            
        full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extrahiere alles nach 'assistant'
        if "assistant" in full_text:
            return full_text.split("assistant")[-1].strip()
        return full_text

    def execute_sql(self, query):
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute(query)
            results = cursor.fetchall()
            conn.close()
            return results
        except Exception as e:
            return f"Fehler bei SQL-Ausführung: {e}"

    def run(self):
        schema = "CREATE TABLE employees (id INTEGER, name TEXT, department TEXT, salary INTEGER, hire_date DATE)"
        print("\n✅ Agent bereit! Tippe 'exit' zum Beenden.")
        
        while True:
            # Hier wartet Colab auf deine Eingabe
            user_input = input("\nDeine Frage an die Datenbank: ")
            
            if user_input.lower() in ["exit", "quit"]:
                print("👋 Bis bald!")
                break
                
            # 1. Denken (SQL generieren)
            sql = self.generate_sql(user_input, schema)
            print(f"🧠 Gedanke (SQL): {sql}")
            
            # 2. Handeln (SQL ausführen)
            data = self.execute_sql(sql)
            
            # 3. Antworten
            print(f"📊 Ergebnis aus DB: {data}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--adapter", type=str, required=True)
    args = parser.parse_args()

    agent = SQLAgent(
        base_model_id="Qwen/Qwen2.5-1.5B-Instruct",
        adapter_id=args.adapter,
        db_path="data/dummy_database.db"
    )
    agent.run()

In [ ]:
!python scripts/setup_db.py

In [ ]:
# Wir evaluieren das Modell aus dem Ordner 'outputs/v1'
!python scripts/evaluate.py \
    --adapter_path "./outputs/v1" \
    --num_samples 50